### Master oficial en Big Data & Data Science - Universidad Internacional de Valencia (VIU)

<font color="darkblue">

-   **Alumno:** Franco Rojas Yucra
-   **TFM:** Sistema para la busqueda de conocimientos en textos de documentos multimedia
-   **Email:** franco.rojasyucra@alumnos.viu.es
-   **Fecha:** Octubre de 2023
<font color="black">

## 04 Entrenamiento de modelo GPT2-Base con fine tuning

Librerías generales

In [1]:
import pandas as pd
import numpy as np
import os
import random
import math
from IPython.display import display, HTML

In [2]:
pd.set_option('display.max_columns',None) #motrar todas las columnas

Estableciendo directorio trabajo

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [4]:
print('Ruta actual: '+os.getcwd())
os.chdir('/content/drive/MyDrive/TFM_VIU_Franco_Rojas')
print('Ruta donde se almacenaran archivos: '+os.getcwd())

Ruta actual: /content
Ruta donde se almacenaran archivos: /content/drive/MyDrive/TFM_VIU_Franco_Rojas


### Configuraciones iniciales para trabajar con LLM

Intalación de librerias para LLM

In [5]:
!pip install -q huggingface_hub datasets evaluate transformers[torch] accelerate -U

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 295.0/295.0 kB 6.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 519.6/519.6 kB 34.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 81.4/81.4 kB 11.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 97.4 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 258.1/258.1 kB 31.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 115.3/115.3 kB 14.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 25.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 16.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.8/7.8 MB 120.0 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 1.3/1.3 MB 81.2 MB/s eta 0:00:00


In [6]:
!apt install git-lfs
!git config --global user.email "franco.rojas.yucra@gmail.com"
!git config --global user.name "franco-rojas"

Reading package lists... Done
Building dependency tree... Done
Reading state information... Done
git-lfs is already the newest version (3.0.2-1ubuntu0.2).
0 upgraded, 0 newly installed, 0 to remove and 18 not upgraded.


Importando librerías para LLM

In [7]:
from datasets import load_dataset
from datasets import ClassLabel
from transformers import AutoTokenizer
from transformers import AutoModelForCausalLM
from transformers import Trainer, TrainingArguments
import accelerate

Loguearse con cuenta de Hugging Face para carga de modelos LLM
##### Cuenta logueada: https://huggingface.co/franco-rojas

In [8]:
from huggingface_hub import notebook_login
notebook_login()

### Lectura de dataset y preparación para entrenamiento

Dataset de transcripciones despues de limpieza

In [9]:
df_transcrip=pd.read_csv('transformed_corpus/conjunto_transcripciones_fin.csv',sep='|')

In [10]:
df_transcrip.head()

,id_doc,tema,transcription_filt2
0,DOC0001,Internet de las cosas,"Bienvenidos a nuestro canal de YouTube, Intern..."
1,DOC0002,Internet de las cosas,Bienvenidos al curso Internet de las Cosas. Mi...
2,DOC0003,Introduccion IA,Hola a todos y bienvenidos a un nuevo video en...
3,DOC0004,Introduccion IA,Hola a todos y bienvenidos a un nuevo video en...
4,DOC0005,Introduccion IA,Hola a todos y bienvenidos a un nuevo video en...


Lectura de dataset con librería datasets

In [11]:
dataset = load_dataset('csv', data_files=['transformed_corpus/conjunto_transcripciones_fin.csv'],delimiter="|", split='train')

Extracting data files:   0%|          | 0/1 [00:00<?, ?it/s]

Generating train split: 0 examples [00:00, ? examples/s]

In [12]:
dataset

Dataset({
    features: ['id_doc', 'tema', 'transcription_filt2'],
    num_rows: 147
})

Separación de documentos: 80% (entrenamiento) y 20% (validación)

In [13]:
datasets = dataset.train_test_split(test_size=0.20)

In [14]:
datasets

DatasetDict({
    train: Dataset({
        features: ['id_doc', 'tema', 'transcription_filt2'],
        num_rows: 117
    })
    test: Dataset({
        features: ['id_doc', 'tema', 'transcription_filt2'],
        num_rows: 30
    })
})

Modelo a utilizar

In [15]:
model_checkpoint = "mrm8488/spanish-gpt2"

Descargando tokenizador de modelo original

In [16]:
tokenizer = AutoTokenizer.from_pretrained(model_checkpoint, use_fast=True)

Tokenización de documentos de acuerdo a modelo original

In [17]:
def tokenize_fn(examples):
    return tokenizer(examples["transcription_filt2"])

In [18]:
df_transcrip.columns

Index(['id_doc', 'tema', 'transcription_filt2'], dtype='object')

In [19]:
tokenized_datasets = datasets.map(tokenize_fn, batched=True, remove_columns=["id_doc","tema","transcription_filt2"])

Map:   0%|          | 0/117 [00:00<?, ? examples/s]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

In [20]:
#La columna con los valores tokenizados y transformados a numerico es "input_ids"
#los valores para la matriz de atención es "attention_mask"
tokenized_datasets.column_names

{'train': ['input_ids', 'attention_mask'],
 'test': ['input_ids', 'attention_mask']}

In [21]:
tokenized_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 117
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask'],
        num_rows: 30
    })
})

Transformación de dataset para entrenamiento

In [22]:
#block_size = tokenizer.model_max_length
block_size = 128

In [23]:
#Función para agrupar textos y definir labels a predecir
def group_texts(examples):
    # Concatenate all texts.
    concatenated_examples = {k: sum(examples[k], []) for k in examples.keys()}
    total_length = len(concatenated_examples[list(examples.keys())[0]])
    total_length = (total_length // block_size) * block_size
    result = {
        k: [t[i : i + block_size] for i in range(0, total_length, block_size)]
        for k, t in concatenated_examples.items()
    }
    result["labels"] = result["input_ids"].copy()
    return result

In [24]:
lm_datasets = tokenized_datasets.map(group_texts,batched=True,batch_size=1000)

Map:   0%|          | 0/117 [00:00<?, ? examples/s]

Map:   0%|          | 0/30 [00:00<?, ? examples/s]

In [25]:
lm_datasets

DatasetDict({
    train: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 1873
    })
    test: Dataset({
        features: ['input_ids', 'attention_mask', 'labels'],
        num_rows: 475
    })
})

### Entrenamiento de modelo

Descargando modelo original desde Hugging Face

In [26]:
model = AutoModelForCausalLM.from_pretrained(model_checkpoint)

Parametros de entrenamiento

In [27]:
training_args = TrainingArguments(
    "finetuned_models",
    #per_device_train_batch_size = 2,
    #per_device_eval_batch_size = 2,
    evaluation_strategy = "epoch",
    learning_rate=2e-5,
    weight_decay=0.01,
    push_to_hub=True,
    push_to_hub_model_id=f"gpt2-base-finetuned-tfmviu"
)

/usr/local/lib/python3.10/dist-packages/transformers/training_args.py:1696: FutureWarning: `--push_to_hub_model_id` is deprecated and will be removed in version 5 of 🤗 Transformers. Use `--hub_model_id` instead and pass the full repo name to this argument (in this case franco-rojas/gpt2-base-finetuned-tfmviu).
  warnings.warn(


In [28]:
trainer = Trainer(
    model=model,
    args=training_args,
    train_dataset=lm_datasets["train"],
    eval_dataset=lm_datasets["test"],
)

Metrica Perplejidad previo a entrenamiento

In [29]:
evaluate_prev=trainer.evaluate()
print(f"Perplexity: {math.exp(evaluate_prev['eval_loss']):.2f}")

Perplexity: 46.23


Entrenamiento del modelo

In [30]:
%%time
trainer.train()

Epoch,Training Loss,Validation Loss
1,No log,3.560417
2,No log,3.530802
3,3.512600,3.523468


CPU times: user 1min 2s, sys: 2.25 s, total: 1min 5s
Wall time: 1min 45s


TrainOutput(global_step=705, training_loss=3.4745015029366133, metrics={'train_runtime': 63.8741, 'train_samples_per_second': 87.97, 'train_steps_per_second': 11.037, 'total_flos': 367049981952000.0, 'train_loss': 3.4745015029366133, 'epoch': 3.0})

Metrica Perplejidad para muestra de evaluación

In [31]:
eval_results = trainer.evaluate()
print(f"Perplexity: {math.exp(eval_results['eval_loss']):.2f}")

Perplexity: 33.90


#### Carga final de modelo y tokenizador a Hugging Face

In [32]:
trainer.push_to_hub()

pytorch_model.bin:   0%|          | 0.00/498M [00:00<?, ?B/s]

'https://huggingface.co/franco-rojas/gpt2-base-finetuned-tfmviu/tree/main/'

In [33]:
tokenizer.push_to_hub('gpt2-base-finetuned-tfmviu')

CommitInfo(commit_url='https://huggingface.co/franco-rojas/gpt2-base-finetuned-tfmviu/commit/de4e30e88c4b737eceef8dd7d60e8638f89cdbbb', commit_message='Upload tokenizer', commit_description='', oid='de4e30e88c4b737eceef8dd7d60e8638f89cdbbb', pr_url=None, pr_revision=None, pr_num=None)